In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from itertools import chain
from pathlib import Path

params = {
    'axes.grid' : True,
    "grid.linestyle": '--',
    "font.family": "serif",
    "font.serif": "Times New Roman",
}

sns.set_style("ticks", params)
sns.set_context("paper", font_scale=1.5)
sns.set_palette("Set2")

In [ ]:
pretrained_model = Path("<path to pretrained 7B flores results>")
traditional_finetune_model = Path("<path to best finetuned 7B model flores results>")
zero_shot_model = Path("<path to best zero-shot adapters 7B model flores results>")
balanced_few_shot_model = Path("<path to best balanced few-shot adapters 7B model flores results>")
unbalanced_few_shot_model = Path("<path to best unbalanced few-shot adapters 7B model flores results>")

In [ ]:
def load_scores(scores_file: Path):
    lines = scores_file.read_text().splitlines()
    scores = {}
    for line in lines:
        key, value = line.split(": ")
        scores[key] = float(value)
    return scores

def load_lp(data_root: Path, lp: str):
    ckpt_dirs = [d for d in (data_root / lp).iterdir() if d.is_dir()]
    lp_results = []
    for ckpt_dir in ckpt_dirs:
        ckpt = ckpt_dir.name
        sys_scores_path = ckpt_dir / "zero_shot_instructions" / "sys_scores.txt"
        scores = load_scores(sys_scores_path)
        lp_results.append({"lp": lp, "Context": "Zero-shot", "Step": int(ckpt), **scores})
        sys_scores_path = ckpt_dir / "few_shot_instructions2" / "sys_scores.txt"
        scores = load_scores(sys_scores_path)
        lp_results.append({"lp": lp, "Context": "Five-shot", "Step": int(ckpt), **scores})
    return lp_results


def load_results(data_root: Path):
    results = []
    lps_dirs = [d for d in data_root.iterdir() if d.is_dir()]
    for lp_dir in lps_dirs:
        lp = lp_dir.name
        results.extend(load_lp(data_root, lp))
    return pd.DataFrame(results)

pretrained_results = load_results(pretrained_model)
pretrained_results["Model"] = "Pretrained"
zero_shot_results = load_results(zero_shot_model)
zero_shot_results["Model"] = "FT w/o few-shot"
balanced_few_shot_results = load_results(balanced_few_shot_model)
balanced_few_shot_results["Model"] = "FT w/ few-shot"
unbalanced_few_shot_results = load_results(unbalanced_few_shot_model)
unbalanced_few_shot_results["Model"] = "Finetuned w ICL (unbalanced)"
#results = pd.concat([zero_shot_results, balanced_few_shot_results, unbalanced_few_shot_results])
results = pd.concat([zero_shot_results, balanced_few_shot_results])
results

In [ ]:
pretrained_results = pretrained_results[pretrained_results["lp"].str.contains("zh")]
pretrained_results

In [ ]:
results = results[results["lp"].str.contains("zh")]
results

In [ ]:
_, ax = plt.subplots(figsize=(5, 3.5))
g = sns.barplot(
    data=results[results["lp"].str.contains("zh")],
    x="Model",
    y="COMET-22",
    hue="Context",
    errorbar=None,
    ax=ax,
)
g.set_xlabel("")
g.set_ylabel("COMET")
plt.ylim(0.74, 0.78)
sns.move_legend(
    g, "lower center",
    bbox_to_anchor=(.5, -.3), ncol=2, title=None, frameon=True,
)
#plt.savefig("figures/chinese_results.pdf", bbox_inches="tight", dpi=200)

In [ ]:
pretrained_root_path = Path("<path to pretrained 7B flores results>")
best_adapters_root_path = Path("<path to best zero-shot adapters 7B model flores results>")
finetune_root_path = Path("<path to best finetuned 7B model flores results>")

def load_scores(scores_file: Path):
    lines = scores_file.read_text().splitlines()
    scores = {}
    for line in lines:
        key, value = line.split(": ")
        scores[key] = float(value)
    return scores

def load_lp(data_root: Path, lp: str, instructions, ckpt: str):
    sys_scores_path = data_root / lp / ckpt / instructions / "sys_scores.txt"
    scores = load_scores(sys_scores_path)
    return {"lp": lp, "Direction": "En-XX" if lp.startswith("en") else "XX-En", **scores}


def load_results(data_root: Path, instructions, ckpt: str):
    results = []
    lps_dirs = [d for d in data_root.iterdir() if d.is_dir()]
    for lp_dir in lps_dirs:
        lp = lp_dir.name
        results.append(load_lp(data_root, lp, instructions, ckpt))
    return pd.DataFrame(results)


pretrained_results = load_results(pretrained_root_path, "few_shot_instructions2", "0")
pretrained_results["Model"] = "Pretrained"
finetune_results = load_results(finetune_root_path, "zero_shot_instructions", "240000")
finetune_results["Model"] = "Finetuned"
best_adapters_results = load_results(best_adapters_root_path, "zero_shot_instructions", "20000")
best_adapters_results["Model"] = "LoRA"

results = pd.concat([pretrained_results, finetune_results, best_adapters_results])
results.rename(columns={"lp": "Language Pair" }, inplace=True)
results["COMET-22"] = results["COMET-22"] * 100
results

In [ ]:
no_zh_results = results[results["Language Pair"].str.contains("zh")]
no_zh_lang_pairs = ["zh-en", "en-zh"]
_, ax = plt.subplots(figsize=(5, 3.5))
g = sns.barplot(
    data=no_zh_results, x="Language Pair", y="COMET-22", hue="Model",
    order=no_zh_lang_pairs,
    #order=["En-XX", "XX-En"],
    hue_order=["Pretrained", "Finetuned", "LoRA"],
    ax=ax,
)
g.legend().set_title("")
g.set_ylabel("COMET")
plt.ylim(60, 85)
sns.move_legend(
    g, "lower center",
    bbox_to_anchor=(.5, -.4), ncol=3, title=None, frameon=True,
)
#plt.savefig("figures/chinese_adapter_vs_finetuning.pdf", bbox_inches="tight", dpi=200)